In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
pd.plotting.register_matplotlib_converters()
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
print("Setup Complete")

In [ ]:
train_filepath = "../input/store-sales-time-series-forecasting/train.csv"
test_filepath = "../input/store-sales-time-series-forecasting/test.csv"
oil_filepath = "../input/store-sales-time-series-forecasting/oil.csv"
holidays_filepath = "../input/store-sales-time-series-forecasting/holidays_events.csv"
transactions_filepath = "../input/store-sales-time-series-forecasting/transactions.csv"
stores_filepath = "../input/store-sales-time-series-forecasting/stores.csv"



In [ ]:
train_data = pd.read_csv(train_filepath, index_col="id")
test_data = pd.read_csv(test_filepath, index_col="date")
oil_data = pd.read_csv(oil_filepath, index_col="date", parse_dates=True)
holidays_data = pd.read_csv(holidays_filepath, index_col="date", parse_dates=True)
transactions_data = pd.read_csv(transactions_filepath, index_col="date")
stores_data = pd.read_csv(stores_filepath, index_col="store_nbr")


In [ ]:
# Convert all 'date' columns into Panda's format:
train_data['date'] = pd.to_datetime(train_data['date'], format = "%Y-%m-%d")
test_data['date'] = pd.to_datetime(test_data['date'], format = "%Y-%m-%d")
oil_data['date'] = pd.to_datetime(oil_data['date'], format = "%Y-%m-%d")
holidays_data['date'] = pd.to_datetime(holidays_data['date'], format = "%Y-%m-%d")
transactions_data['date'] = pd.to_datetime(transactions_data['date'], format ="%Y-%m-%d")

In [ ]:
train_data.head()

In [ ]:
test_data.head()


In [ ]:
plt.figure(figsize=(16,6))
plt.title('Sales per product family')
sns.barplot(x=train_data.sales, y=train_data.family)
plt.xlabel('Sales')
plt.ylabel('Product Family')



In [ ]:
oil_data.head()

In [ ]:
plt.figure(figsize=(16,6))
sns.lineplot(data=oil_data.dcoilwtico, label="Oil price")

In [ ]:
plt.figure(figsize=(16,6))
sns.scatterplot(x=train_data['onpromotion'], y=train_data['sales'])

In [ ]:
#Merge Data
train_data_m1 = train_data.merge(oil_data, on = 'date', how = 'left')
train_data_m1

In [ ]:
train_data_m1.head()

In [ ]:
train_data_m1.tail()

In [ ]:
# Grouping data set by date and average oil price per day
oil_price = train_data_m1.groupby('date').dcoilwtico.mean()
oil_price

In [ ]:
# grouping data set by date and deriving ave sales per day
ave_sales = train_data_m1.groupby('date').sales.mean()
ave_sales

In [ ]:
#converting data to data frame
ave_sales.to_frame()

In [ ]:
#converting data to data frame
oil_price.to_frame()

In [ ]:
#Using concat to merge columns with similar id
oil_ave_sales = pd.concat([ave_sales, oil_price], axis=1)
oil_ave_sales

In [ ]:
plt.figure(figsize=(16,6))
sns.scatterplot(x=oil_ave_sales['dcoilwtico'], y=oil_ave_sales['sales'])



In [ ]:
plt.figure(figsize=(16,6))
sns.regplot(x=oil_ave_sales['dcoilwtico'], y=oil_ave_sales['sales'])

In [ ]:
#Using concat to merge columns with similar id
oil_ave_sales1 = pd.concat([ave_sales, oil_price], axis=1)
oil_ave_sales1

In [ ]:
#filling NaN with values above and below the cell
oil_ave_sales1['dcoilwtico'] = oil_ave_sales1['dcoilwtico'].fillna(method='ffill')
oil_ave_sales1['dcoilwtico'] = oil_ave_sales1['dcoilwtico'].fillna(method='bfill')
oil_ave_sales1

In [ ]:
plt.figure(figsize=(16,6))
sns.regplot(x=oil_ave_sales1['dcoilwtico'], y=oil_ave_sales1['sales'])

In [ ]:
# add dummy to holidays

holidays_data['Dummy Holiday'] = '1'
holidays_data

In [ ]:
#"groupby" Holidays_data by Dummy Holiday

dummy_holidays = holidays_data["Dummy Holiday"]
dummy_holidays

In [ ]:
dummy_holidays.to_frame()

In [ ]:
final_df = oil_ave_sales1.merge(dummy_holidays, on = 'date', how = 'left')
final_df

In [ ]:
print(final_df.to_string())

In [ ]:
#replace NaN with 0

final_df['Dummy Holiday'] = final_df['Dummy Holiday'].fillna(0)
final_df

In [ ]:
# add 'on promotion' to final_df
# onpromotion has number of products on sale that  day

ave_promotion = train_data_m1.groupby('date').onpromotion.mean()
ave_promotion

In [ ]:
print(ave_promotion.to_string())

In [ ]:
ave_promotion.to_frame()

In [ ]:
#Use merge to add a column of data to the set

final_df = final_df.merge(ave_promotion, on = 'date', how = 'left')
final_df

In [ ]:
#Regression Plot: Overall Sales and Explanatory Variables
plt.figure(figsize=(16,6))
plt.title('Average Sales and Explanatory Variable')
sns.regplot(x=final_df['onpromotion'], y=final_df['sales'])

In [ ]:
#Group by multiple columns and averaging sales
ave_family_sales = train_data_m1.groupby(["date","family"]).sales.mean()
ave_family_sales

In [ ]:
ave_family_sales.to_frame()

In [ ]:
#Extracting Select Data from a DF

grocery_sales = train_data_m1.loc[train_data_m1.family == 'GROCERY I']
grocery_sales

In [ ]:
# Average Grocery I Sales, grouped by date
ave_grocery_sales = grocery_sales.groupby('date').sales.mean()
ave_grocery_sales

In [ ]:
oil_ave_grocery_sales = pd.concat([ave_grocery_sales, oil_price], axis = 1)
oil_ave_grocery_sales

In [ ]:
oil_ave_grocery_sales['dcoilwtico'] = oil_ave_grocery_sales['dcoilwtico'].fillna(method='ffill')
oil_ave_grocery_sales['dcoilwtico'] = oil_ave_grocery_sales['dcoilwtico'].fillna(method='bfill')
oil_ave_grocery_sales

In [ ]:
# Merge holiday dummy data to the set

grocery_final_df = oil_ave_grocery_sales.merge(dummy_holidays, on = 'date', how = 'left')
grocery_final_df

In [ ]:
#replace NaN with 0

grocery_final_df['Dummy Holiday'] = grocery_final_df['Dummy Holiday'].fillna(0)
grocery_final_df

In [ ]:
#derive onpromotion under Grocery I

ave_grocery_promo = grocery_sales.groupby('date').onpromotion.mean()
ave_grocery_promo

In [ ]:
grocery_final_df = grocery_final_df.merge(ave_grocery_promo, on = 'date', how = 'left')
grocery_final_df



In [ ]:
#Regression Plot: Grocery Sales and Explanatory Variables
plt.figure(figsize=(16,6))
plt.title('Average Grocery Sales and Explanatory Variable')
sns.regplot(x=grocery_final_df['onpromotion'], y=grocery_final_df['sales'])

In [ ]:
#Regression OLS -- Average Total Sales and explanatory variables

# Step 1: Create X and Y data matrices
X = final_df.drop(['sales'], axis = 1)
Y = final_df.drop(['dcoilwtico', 'onpromotion', 'Dummy Holiday'], axis = 1)

In [ ]:
Y

In [ ]:
X

In [ ]:
Y = pd.DataFrame(Y)
Y

In [ ]:
X = pd.DataFrame(X)
X

In [ ]:
# Step 2: Import Library

from sklearn.model_selection import train_test_split

In [ ]:
# Step 3: Perfrom 70/30 data split

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.3)

In [ ]:
# Step 4: Check data dimension

X_train.shape, Y_train.shape


In [ ]:
X_test.shape, Y_test.shape

In [ ]:
# Step 4: Import library for Linear Regression Model

from sklearn import linear_model
from sklearn.metrics import mean_squared_error, r2_score

In [ ]:
# Another method for Linear Regression follows

In [ ]:
import statsmodels.api as sm
from statsmodels.sandbox.regression.predstd import wls_prediction_std

In [ ]:
X = sm.add_constant(X)
X

In [ ]:
# print regression result

est = sm.OLS(Y.astype(float), X.astype(float)).fit()
est
print(est.summary())

In [ ]:
# OLS Regression for Ave Grocery Sales

Y_grocery = grocery_final_df.drop(['dcoilwtico', 'onpromotion', 'Dummy Holiday'], axis = 1)
Y_grocery

In [ ]:
X_grocery = grocery_final_df.drop(['sales'], axis = 1)
X_grocery

In [ ]:
X_grocery = sm.add_constant(X_grocery)
X_grocery

In [ ]:
#Regression result for Grocery Sales

model_g = sm.OLS(Y_grocery.astype(float), X_grocery.astype(float))
result = model_g.fit()
print(result.summary())

In [ ]:
#using natural log of oil price

final_df['log oil price'] = np.log(final_df['dcoilwtico'])
final_df

In [ ]:
# Step 1: Create X and Y data matrices
x = final_df.drop(['sales', 'dcoilwtico'], axis = 1)
y = final_df.drop(['dcoilwtico', 'onpromotion', 'Dummy Holiday', 'log oil price'], axis = 1)

In [ ]:
y

In [ ]:
x

In [ ]:
y = pd.DataFrame(y)
y

In [ ]:
x = pd.DataFrame(x)
x

In [ ]:
x = sm.add_constant(x)
x

In [ ]:
#OLS with log oil price

est = sm.OLS(y.astype(float), x.astype(float)).fit()
est
print(est.summary())

In [ ]:
# GROCERY SALES -- using natural log of oil price

grocery_final_df['log oil price'] = np.log(grocery_final_df['dcoilwtico'])
grocery_final_df

In [ ]:
# Step 1: Create X and Y data matrices
x_grocery = grocery_final_df.drop(['sales', 'dcoilwtico'], axis = 1)
y_grocery = grocery_final_df.drop(['dcoilwtico', 'onpromotion', 'Dummy Holiday', 'log oil price'], axis = 1)

In [ ]:
x_grocery

In [ ]:
y_grocery

In [ ]:
y_grocery = pd.DataFrame(y_grocery)
y_grocery

In [ ]:
x_grocery = pd.DataFrame(x_grocery)
x_grocery

In [ ]:
x_grocery = sm.add_constant(x_grocery)
x_grocery

In [ ]:
#OLS with log oil price

est = sm.OLS(y_grocery.astype(float), x_grocery.astype(float)).fit()
est
print(est.summary())

In [ ]:
# Random Forest for Regression

from sklearn.preprocessing import StandardScaler

sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [ ]:
from sklearn.ensemble import RandomForestRegressor

regressor = RandomForestRegressor(n_estimators=100, random_state=0)
regressor.fit(X_train, Y_train)
y_pred = regressor.predict(X_test)

In [ ]:
Y_test

In [ ]:
y_pred

In [ ]:
print(y_pred.shape)

In [ ]:
from sklearn import metrics

print('Mean Absolute Error:', metrics.mean_absolute_error(Y_test, y_pred))
print('Mean Squared Error:', metrics.mean_squared_error(Y_test, y_pred))
print('R2:', metrics.r2_score(Y_test, y_pred))
print('Explained Variance:', metrics.explained_variance_score(Y_test, y_pred))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(Y_test, y_pred)))

In [ ]:
# Random Forest for Grocery Sales
# Step 3: Perfrom 70/30 data split

X_grocery_train, X_grocery_test, Y_grocery_train, Y_grocery_test = train_test_split(X_grocery, Y_grocery, test_size = 0.3)

In [ ]:
from sklearn.preprocessing import StandardScaler

sc = StandardScaler()
X_grocery_train = sc.fit_transform(X_grocery_train)
X_grocery_test = sc.transform(X_grocery_test)

In [ ]:
from sklearn.ensemble import RandomForestRegressor

regressor = RandomForestRegressor(n_estimators=200, random_state=0)
regressor.fit(X_grocery_train, Y_grocery_train)
y_grocery_pred = regressor.predict(X_grocery_test)

In [ ]:
y_grocery_pred

In [ ]:
print(y_grocery_pred.shape)

In [ ]:
Y_grocery_test

In [ ]:
from sklearn import metrics

print('Mean Absolute Error:', metrics.mean_absolute_error(Y_grocery_test, y_grocery_pred))
print('Mean Squared Error:', metrics.mean_squared_error(Y_grocery_test, y_grocery_pred))
print('R2:', metrics.r2_score(Y_grocery_test, y_grocery_pred))
print('Explained Variance:', metrics.explained_variance_score(Y_grocery_test, y_grocery_pred))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(Y_grocery_test, y_grocery_pred)))

In [ ]:
# load reshaped transactions_data

transact_filepath = "../input/transaction-data-final/transactions_data_F.csv"

In [ ]:
transact_data = pd.read_csv(transact_filepath, index_col="Id")

In [ ]:
transact_data.head()

In [ ]:
transact_data['date'] = pd.to_datetime(transact_data['date'], format ="%Y-%m-%d")

In [ ]:
transact_data.mean()

In [ ]:
transact_data = transact_data.fillna(transact_data.mean())
transact_data

In [ ]:
final_df2 = final_df.merge(transact_data, on = "date", how = 'left')
final_df2

In [ ]:
final_df3 = final_df2.set_index('date')
final_df3

In [ ]:
final_df3.isnull().sum().sum()

In [ ]:
final_df3 = final_df3.fillna(final_df3.mean())
final_df3

In [ ]:
final_df3.isnull().sum().sum()

In [ ]:
# Step 1: Create X and Y data matrices
x_fin = final_df3.drop(['sales', 'log oil price'], axis = 1)
y_fin = final_df3['sales']

In [ ]:
x_fin

In [ ]:
y_fin

In [ ]:
y_fin = pd.DataFrame(y_fin)
y_fin

In [ ]:
x_fin = pd.DataFrame(x_fin)
x_fin

In [ ]:
# Step 2: Import Library

from sklearn.model_selection import train_test_split

In [ ]:
# Step 3: Perfrom 70/30 data split

x_fin_train, x_fin_test, y_fin_train, y_fin_test = train_test_split(x_fin, y_fin, test_size = 0.3)

In [ ]:
# Step 4: Import library for Linear Regression Model

from sklearn import linear_model
from sklearn.metrics import mean_squared_error, r2_score

In [ ]:
from sklearn.preprocessing import StandardScaler

sc = StandardScaler()
x_fin_train = sc.fit_transform(x_fin_train)
x_fin_test = sc.transform(x_fin_test)

In [ ]:
x_fin_train

In [ ]:
x_fin_test

In [ ]:
from sklearn.ensemble import RandomForestRegressor

regressor = RandomForestRegressor(n_estimators=20, random_state=0)
regressor.fit(x_fin_train, y_fin_train)
y_fin_pred = regressor.predict(x_fin_test)

In [ ]:
from sklearn import metrics

print('Mean Absolute Error:', metrics.mean_absolute_error(y_fin_test, y_fin_pred))
print('Mean Squared Error:', metrics.mean_squared_error(y_fin_test, y_fin_pred))
print('R2:', metrics.r2_score(y_fin_test, y_fin_pred))
print('Explained Variance:', metrics.explained_variance_score(y_fin_test, y_fin_pred))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_fin_test, y_fin_pred)))